In [1]:
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 913 µs (started: 2021-11-15 12:47:58 -08:00)


# Options Apps Project

## Goal
Creating a suite of equity option web apps which expands on, and was inspired by, [Harshit Tyagi’s](https://www.linkedin.com/in/tyagiharshit/) [article](https://www.kdnuggets.com/2021/09/-structured-financial-newsfeed-using-python-spacy-and-streamlit.html) on building a financial newsfeed app.

## Skills Involved
Web scraping, NLP, options price modeling, predictive modeling of volitility, and building apps with Streamlit.

## Imports

In [2]:
import sys, os, math
import requests
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import time

#  setting path
gparent = os.path.join(os.pardir)
sys.path.append(gparent)

from src import helper_functions as h

time: 3.7 s (started: 2021-11-15 12:47:58 -08:00)


## requests: Grabbing Headlines

In [3]:
# grabbing headlines
r = requests.get("http://feeds.marketwatch.com/marketwatch/topstories/")
r

<Response [200]>

time: 235 ms (started: 2021-11-15 12:48:02 -08:00)


## bs4: Saving Headlines

In [4]:
# saving headlines to a list
soup = BeautifulSoup(r.content, features='lxml')
headlines = soup.findAll('title')
print(f'{type(headlines)}\n')
print(headlines)

<class 'bs4.element.ResultSet'>

[<title>MarketWatch.com - Top Stories</title>, <title>MarketWatch.com - Top Stories</title>, <title>: Hershey’s giant 3.4-pound Reese’s Peanut Butter Cup ‘pies’ for Thanksgiving sold out in less than two hours</title>, <title>Key Words: Ron Wyden’s hedge fund-manager son walks back criticism of dad — just in time for Thanksgiving</title>, <title>: Older U.S. workers are ‘unretiring’</title>, <title>The Ratings Game: CrowdStrike stock drops more than 10% as Morgan Stanley starts coverage with sell rating</title>, <title>Key Words: The ‘dirty little secret’ is that many Americans can afford inflation: NBC’s Stephanie Ruhle</title>, <title>: The pandemic changed some small business owners’ attitudes about paid family leave</title>, <title>The Ratings Game: Take-Two stock falls more than 5% after Rockstar Games has ‘Grand Theft Auto’ trouble</title>, <title>Futures Movers: U.S. oil futures finish higher, shake off losses from fears of a possible Strategic P

### Grabbing Headline String
Grabbing headline and printing headline and isolated text.

In [5]:
# grabbing test headline
test_headline = headlines[2]
print(f'{test_headline}\n')
print(test_headline.text)

<title>: Hershey’s giant 3.4-pound Reese’s Peanut Butter Cup ‘pies’ for Thanksgiving sold out in less than two hours</title>

: Hershey’s giant 3.4-pound Reese’s Peanut Butter Cup ‘pies’ for Thanksgiving sold out in less than two hours
time: 953 µs (started: 2021-11-15 12:48:02 -08:00)


## spaCy: Tests
Testing tokenization and name extraction on a single string.

### Loading Model

In [6]:
# loading spacy model
nlp = spacy.load('en_core_web_sm')

time: 523 ms (started: 2021-11-15 12:48:02 -08:00)


### Tokenizing String
Tokenizing the test headline.

In [7]:
# checking the test case
processed_hline = nlp(test_headline.text)
print(f'{test_headline}\n')
for token in processed_hline:
  print(token)

<title>: Hershey’s giant 3.4-pound Reese’s Peanut Butter Cup ‘pies’ for Thanksgiving sold out in less than two hours</title>

:
Hershey
’s
giant
3.4-pound
Reese
’s
Peanut
Butter
Cup
‘
pies
’
for
Thanksgiving
sold
out
in
less
than
two
hours
time: 13.1 ms (started: 2021-11-15 12:48:03 -08:00)


## Saving List of Tokenized Headlines

In [8]:
processed_hlines = [nlp(headlines[i].text) for i in range(len(headlines))]
for line in processed_hlines:
    print(line)

MarketWatch.com - Top Stories
MarketWatch.com - Top Stories
: Hershey’s giant 3.4-pound Reese’s Peanut Butter Cup ‘pies’ for Thanksgiving sold out in less than two hours
Key Words: Ron Wyden’s hedge fund-manager son walks back criticism of dad — just in time for Thanksgiving
: Older U.S. workers are ‘unretiring’
The Ratings Game: CrowdStrike stock drops more than 10% as Morgan Stanley starts coverage with sell rating
Key Words: The ‘dirty little secret’ is that many Americans can afford inflation: NBC’s Stephanie Ruhle
: The pandemic changed some small business owners’ attitudes about paid family leave
The Ratings Game: Take-Two stock falls more than 5% after Rockstar Games has ‘Grand Theft Auto’ trouble
Futures Movers: U.S. oil futures finish higher, shake off losses from fears of a possible Strategic Petroleum Reserve release
The Ratings Game: EVgo stock tumbles after BofA says sell, citing recent EV ‘charging euphoria’ and pending lockup expiry
: Biden to sign bipartisan infrastruct

## Getting Org Names From Headlines
Visualizing named entities (real world objects) in the headlines and creating a set of organiziations.

In [9]:
# pulling company name tokens from headlines
companies = []
for title in processed_hlines:
    doc = nlp(title.text)
    if len(doc.ents) != 0:
        displacy.render(doc, style='ent')
    else:
        pass
    for token in doc.ents:
        if token.label_ == 'ORG':
            companies.append(token.text)
        else:
            pass
companies = set(companies)
print(companies)

{'Morgan Stanley', 'EV', 'Strategic Petroleum Reserve', 'NBC'}
time: 115 ms (started: 2021-11-15 12:48:03 -08:00)


## Scraping S&P 500 Stock Table w/Requests & BeautifulSoup

In [10]:
# scraping S&P wikipedia page
r = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# parsing the html
soup = BeautifulSoup(r.text, 'lxml')

# extracting the table
table = soup.find('table', {'class': 'wikitable sortable'})

# printing row with first stock
print(table.findAll('tr')[1:2])

[<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M</a></td>
<td><a class="external text" href="https://www.sec.gov/edgar/browse/?CIK=66740" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td>
<td>1976-08-09</td>
<td>0000066740</td>
<td>1902
</td></tr>]
time: 599 ms (started: 2021-11-15 12:48:03 -08:00)


### Ticker Symbols
Grabbing the ticker symbol from the first cell of each row.

In [11]:
# making list of symbols
symbols = [row.findAll('td')[0].text for row in table.findAll('tr')[1:]]

# checking length
print(f'List Length: {len(symbols)} \n')

# checking first 5 symbols
print(f'First five symbols: {symbols[:5]}')

List Length: 505 

First five symbols: ['MMM\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n']
time: 15.4 ms (started: 2021-11-15 12:48:04 -08:00)


The length of the list looks good, but we have extra characters we need to strip from the symbols.

In [12]:
# stripping new line character from the strings 
symbols = list(map(lambda s: s.strip(), symbols))

# checking first 5 symbols
print(symbols[:5])

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']
time: 931 µs (started: 2021-11-15 12:48:04 -08:00)


Looks good. We can apply the same technique to the second cell of the row to scrape and save the company name, and to the fifth cell to grab the company industry. 

### Names
Grabbing the company name from the second cell of each row.

In [13]:
# making list of names
names = [row.findAll('td')[1].text for row in table.findAll('tr')[1:]]

# checking first five names
print(names[:5])

['3M', 'Abbott Laboratories', 'AbbVie', 'Abiomed', 'Accenture']
time: 14.2 ms (started: 2021-11-15 12:48:04 -08:00)


## S&P 500 Dataframe
Creating a data frame of stocks in the S&P 500 index.

In [14]:
# making a data dictionary
data = {'Company Name': names, 'Symbol': symbols}

# creating data frame from the data
stocks_df = pd.DataFrame.from_dict(data)

# checking shape and first five rows
print(stocks_df.shape)
stocks_df.head()

(505, 2)


,Company Name,Symbol
0,3M,MMM
1,Abbott Laboratories,ABT
2,AbbVie,ABBV
3,Abiomed,ABMD
4,Accenture,ACN


time: 8.2 ms (started: 2021-11-15 12:48:04 -08:00)


### Checking yf Stock Info Dictionary Keys
Checking the dictionary keys available for pulling stock information.

In [15]:
# instantiating a ticker object
ACN = yf.Ticker('ACN')

time: 2.72 ms (started: 2021-11-15 12:48:04 -08:00)


In [16]:
# checking keys
ACN.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskR

time: 3.12 s (started: 2021-11-15 12:48:04 -08:00)


### Data for Stocks in the News
Creating a data frame of price and dividend information for S&P 500 stocks in the news in the following steps:

- Create a data dictionary from the list of companies in the headlines.

- Create a data frame from data dictionary.

In [17]:
# creating empty stock info dictionary
stock_data = {
    'Company': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    '52wkHigh': [],
    '52wkLow': [],
    'dividendRate': []
    
}

# loading stocks from s&p dataframe and appending data from yf
for company in companies:

    try:
        if stocks_df['Company Name'].str.contains(company).sum():
            symbol = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Symbol'].values[0]
            org_name = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Company Name'].values[0]
            stock_data['Company'].append(org_name)
            stock_data['Symbol'].append(symbol)
            stock_info = yf.Ticker(symbol).info
            stock_data['currentPrice'].append(stock_info['currentPrice'])
            stock_data['dayHigh'].append(stock_info['dayHigh'])
            stock_data['dayLow'].append(stock_info['dayLow'])
            stock_data['52wkHigh'].append(stock_info['fiftyTwoWeekHigh'])
            stock_data['52wkLow'].append(stock_info['fiftyTwoWeekLow'])            
            
            # converting dividend None types to floats
            dividend = stock_info['dividendRate']
            if dividend != None:
                dividend = dividend
            else:
                dividend = 0
            stock_data['dividendRate'].append(dividend)
        else:
            pass
    except:
        pass

time: 2.57 s (started: 2021-11-15 12:48:07 -08:00)


In [18]:
# checking dict
stock_data

{'Company': ['Morgan Stanley'],
 'Symbol': ['MS'],
 'currentPrice': [100.19],
 'dayHigh': [100.8],
 'dayLow': [100.01],
 '52wkHigh': [105.95],
 '52wkLow': [57.03],
 'dividendRate': [2.8]}

time: 2.19 ms (started: 2021-11-15 12:48:10 -08:00)


### Data Frame of S&P 500 Stocks in the News

In [19]:
in_the_news = pd.DataFrame(stock_data)
in_the_news.head()

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate
0,Morgan Stanley,MS,100.19,100.8,100.01,105.95,57.03,2.8


time: 7.22 ms (started: 2021-11-15 12:48:10 -08:00)


In [20]:
# checking Dtypes
in_the_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       1 non-null      object 
 1   Symbol        1 non-null      object 
 2   currentPrice  1 non-null      float64
 3   dayHigh       1 non-null      float64
 4   dayLow        1 non-null      float64
 5   52wkHigh      1 non-null      float64
 6   52wkLow       1 non-null      float64
 7   dividendRate  1 non-null      float64
dtypes: float64(6), object(2)
memory usage: 192.0+ bytes
time: 5.88 ms (started: 2021-11-15 12:48:10 -08:00)


## Individual Stock Price Helper Function
Checking the helper function.

In [21]:
h.prices('MMM')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,3M Company,MMM,182.9426,185,182.93,208.95,163.38,5.92


time: 2.9 s (started: 2021-11-15 12:48:10 -08:00)


In [22]:
h.prices('AAPL')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,Apple Inc.,AAPL,149.8619,151.88,149.43,157.26,112.59,0.88


time: 2.85 s (started: 2021-11-15 12:48:13 -08:00)


## Options Markets Tests

In [23]:
# creating a fuction to display available expirations
def get_expirations(symbol : str):
    stock = yf.Ticker(symbol)
    return stock.options

time: 840 µs (started: 2021-11-15 12:32:16 -08:00)


In [24]:
get_expirations('AAPL')

('2021-11-19',
 '2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-02-18',
 '2022-03-18',
 '2022-04-14',
 '2022-05-20',
 '2022-06-17',
 '2022-09-16',
 '2023-01-20',
 '2023-03-17',
 '2023-06-16',
 '2023-09-15',
 '2024-01-19')

time: 220 ms (started: 2021-11-15 12:32:56 -08:00)


In [25]:
get_expirations('MMM')

('2021-11-19',
 '2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-04-14',
 '2022-06-17',
 '2022-07-15',
 '2023-01-20',
 '2024-01-19')

time: 166 ms (started: 2021-11-15 12:33:22 -08:00)
